In [ ]:

# Mount Google Drive to access your saved .npy files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# model_arch.py - PHASE A: MODEL CONSTRUCTION AND PRE-TRAINING (Colab/Checkpoint Adapted)
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, GRU, Dense, Dropout, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import os
import glob
import re

In [ ]:
# --- Configuration (ADJUST THIS PATH!) ---
DRIVE_BASE_PATH = '/content/drive/MyDrive/IMD PROJECT' # <-- CHECK THIS PATH
SEQUENCES_DATA_PATH = DRIVE_BASE_PATH
MODELS_PATH = os.path.join(DRIVE_BASE_PATH, 'models')

# Model Hyperparameters
SEQ_LENGTH = 30     # Input sequence length (time steps)
N_FEATURES = 8      # Number of features (F1 to F7, F9)
N_CLASSES = 3       # Target classes (0: Down, 1: Flat, 2: Up)
GRU_UNITS = 128
BATCH_SIZE = 64
EPOCHS = 200

# --- Custom Checkpoint Callback ---

class EpochCheckpoint(Callback):
    """Saves the model weights every `save_freq` epochs."""
    def __init__(self, filepath, save_freq=10, verbose=0):
        super(EpochCheckpoint, self).__init__()
        self.filepath = filepath
        self.save_freq = save_freq
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            filename = self.filepath.format(epoch=epoch + 1)
            self.model.save(filename, overwrite=True)
            if self.verbose > 0:
                print(f'\nEpoch {epoch + 1}: Checkpoint saved to {filename}')



In [ ]:
ls

drive/  sample_data/


In [ ]:

# --- Model Definition ---

def build_gru_attention_model(input_shape: tuple, gru_units: int, n_classes: int) -> Model:
    """Constructs the GRU-Attention Hybrid model."""
    print("Building GRU-Attention Model...")

    inputs = Input(shape=input_shape, name='Input_Layer')

    gru_out = GRU(
        units=gru_units,
        return_sequences=True,
        kernel_initializer='he_normal',
        name='GRU_Encoder'
    )(inputs)
    gru_out = Dropout(0.3, name='GRU_Dropout')(gru_out)

    attention_output = Attention(name='Attention_Mechanism')(
        [gru_out, gru_out]
    )

    context_vector = tf.keras.layers.GlobalAveragePooling1D(name='Context_Vector_Pool')(attention_output)

# 5. Output Classifier: Dense layers for the 3-class prediction
    dense_1 = Dense(64, activation='relu', name='Dense_Hidden')(context_vector)
    dense_1 = Dropout(0.3, name='Dense_Dropout')(dense_1)

    outputs = Dense(n_classes, activation='softmax', name='Output_Classifier')(dense_1)

    model = Model(inputs=inputs, outputs=outputs, name='GRU_Attention_Pretrain_Model')

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [ ]:

def get_latest_checkpoint(path: str, prefix: str = 'epoch-') -> tuple[str | None, int]:
    """Checks for the latest epoch checkpoint and returns its path and epoch number."""
    search_pattern = os.path.join(path, f'{prefix}*.h5')
    all_checkpoints = glob.glob(search_pattern)

    if not all_checkpoints:
        return None, 0

    def get_epoch_number(filename):
        match = re.search(r'epoch-(\d+)\.h5$', filename)
        return int(match.group(1)) if match else 0

    latest_checkpoint = max(all_checkpoints, key=get_epoch_number)
    start_epoch = get_epoch_number(latest_checkpoint)

    return latest_checkpoint, start_epoch

In [ ]:

def run_pre_training():
    print("--- Starting Phase A: Model Pre-training on Colab GPU ---")

    # Ensure model save directory exists on Drive
    os.makedirs(MODELS_PATH, exist_ok=True)

    # --- 1. Load Data ---
    X_path = os.path.join(SEQUENCES_DATA_PATH, 'X_pretrain.npy')
    y_path = os.path.join(SEQUENCES_DATA_PATH, 'y_pretrain.npy')

    if not os.path.exists(X_path) or not os.path.exists(y_path):
        print("❌ Error: Sequences not found. Run sequence_prep.py first.")
        return

    X_pretrain = np.load(X_path)
    y_pretrain = np.load(y_path)
    y_pretrain = y_pretrain.astype(int)

    print(f"✅ Data Loaded: X shape {X_pretrain.shape}, y shape {y_pretrain.shape}")

    # --- 2. Check for Resumption Point ---

    input_shape = (SEQ_LENGTH, N_FEATURES)
    start_epoch = 0

    # Check for latest full epoch checkpoint
    latest_checkpoint_path, start_epoch = get_latest_checkpoint(MODELS_PATH)

    if latest_checkpoint_path:
        print(f"🔄 Resuming training from checkpoint: {latest_checkpoint_path}")
        print(f"Starting epoch: {start_epoch + 1}")
        model = load_model(latest_checkpoint_path)
    else:
        print("🆕 No checkpoint found. Starting training from scratch (Epoch 1).")
        model = build_gru_attention_model(input_shape, GRU_UNITS, N_CLASSES)

    model.summary()

    # --- 3. Define Robust Callbacks ---

    # 3a. Best Model Checkpoint (Saves ONLY the best weights based on validation loss)
    best_model_path = os.path.join(MODELS_PATH, 'IT_GRU_Pretrain_Best.h5')
    best_checkpoint = ModelCheckpoint(
        best_model_path,
        monitor='val_loss',
        save_best_only=True,
        mode='min',
        verbose=1
    )

    # 3b. Regular Epoch Checkpoint (Saves model every 10 epochs, regardless of performance)
    regular_checkpoint_path = os.path.join(MODELS_PATH, 'epoch-{epoch:02d}.h5')
    regular_checkpoint = EpochCheckpoint(
        regular_checkpoint_path,
        save_freq=10,
        verbose=1
    )

    # 3c. Early Stopping (Prevents running too long)
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )

    callbacks_list = [best_checkpoint, regular_checkpoint, early_stopping]

    # --- 4. Train Model ---
    print("\nStarting Training...")
    history = model.fit(
        X_pretrain,
        y_pretrain,
        epochs=EPOCHS,
        initial_epoch=start_epoch, # Starts training from the epoch after the last checkpoint
        batch_size=BATCH_SIZE,
        validation_split=0.15,
        callbacks=callbacks_list,
        verbose=1
    )

    print("\n--- Phase A: Pre-training Complete ---")
    print(f"🎉 Final best model weights are saved to: {best_model_path}")

if __name__ == '__main__':
    run_pre_training()

--- Starting Phase A: Model Pre-training on Colab GPU ---
✅ Data Loaded: X shape (26877, 30, 8), y shape (26877,)
🆕 No checkpoint found. Starting training from scratch (Epoch 1).
Building GRU-Attention Model...


Model: "GRU_Attention_Pretrain_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Layer         │ (None, 30, 8)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_Encoder (GRU)   │ (None, 30, 128)   │     52,992 │ Input_Layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ GRU_Dropout         │ (None, 30, 128)   │          0 │ GRU_Encoder[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Attention_Mechanism │ (None, 30, 128)   │          0 │ GRU_Dropout[0][0… │
│ (Attention)         │                   │            │ GRU_Dropout[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Context_Vector_Pool │ (None, 128)       │          0 │ Attention_Mechan… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Hidden        │ (None, 64)        │      8,256 │ Context_Vector_P… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense_Dropout       │ (None, 64)        │          0 │ Dense_Hidden[0][… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Output_Classifier   │ (None, 3)         │        195 │ Dense_Dropout[0]… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 61,443 (240.01 KB)

 Trainable params: 61,443 (240.01 KB)

...

357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3925 - loss: 1.0808 - val_accuracy: 0.4010 - val_loss: 1.0600
Epoch 20/200
350/357 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3918 - loss: 1.0765
Epoch 20: val_loss did not improve from 1.05997



Epoch 20: Checkpoint saved to /content/drive/MyDrive/IMD PROJECT/models/epoch-20.h5
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.3918 - loss: 1.0766 - val_accuracy: 0.3963 - val_loss: 1.0631
Epoch 21/200
351/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3843 - loss: 1.0798
Epoch 21: val_loss did not improve from 1.05997
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3844 - loss: 1.0798 - val_accuracy: 0.4005 - val_loss: 1.0610
Epoch 22/200
352/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3924 - loss: 1.0786
Epoch 22: val_loss did not improve from 1.05997
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.3924 - loss: 1.0786 - val_accuracy: 0.4008 - val_loss: 1.0607
Epoch 23/200
356/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3952 - loss: 1.0766
Epoch 23: val_loss did not improve from 1.05997
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.3952 - loss: 1.0767 - val_accuracy: 0.3963 - val_loss: 1.0611
Epoch 24/200
353/357 ━━━━━━━━━━━━━━━━━

357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3966 - loss: 1.0779 - val_accuracy: 0.3951 - val_loss: 1.0591
Epoch 26/200
354/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3956 - loss: 1.0766
Epoch 26: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3956 - loss: 1.0766 - val_accuracy: 0.4048 - val_loss: 1.0608
Epoch 27/200
355/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3889 - loss: 1.0774
Epoch 27: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3890 - loss: 1.0774 - val_accuracy: 0.3978 - val_loss: 1.0597
Epoch 28/200
352/357 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4016 - loss: 1.0753
Epoch 28: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.4015 - loss: 1.0753 - val_accuracy: 0.3963 - val_loss: 1.0598
Epoch 29/200
351/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3946 - loss: 1.0727
Epoch 29: val_loss did not improve 


Epoch 30: Checkpoint saved to /content/drive/MyDrive/IMD PROJECT/models/epoch-30.h5
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3927 - loss: 1.0752 - val_accuracy: 0.3983 - val_loss: 1.0614
Epoch 31/200
350/357 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3979 - loss: 1.0743
Epoch 31: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3979 - loss: 1.0743 - val_accuracy: 0.4055 - val_loss: 1.0644
Epoch 32/200
356/357 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4020 - loss: 1.0725
Epoch 32: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4020 - loss: 1.0725 - val_accuracy: 0.3926 - val_loss: 1.0614
Epoch 33/200
351/357 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3981 - loss: 1.0728
Epoch 33: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.3981 - loss: 1.0729 - val_accuracy: 0.3961 - val_loss: 1.0658
Epoch 34/200
355/357 ━━━━━━━━━━━━━━━━━━


Epoch 40: Checkpoint saved to /content/drive/MyDrive/IMD PROJECT/models/epoch-40.h5
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.4034 - loss: 1.0688 - val_accuracy: 0.3973 - val_loss: 1.0652
Epoch 41/200
349/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3994 - loss: 1.0701
Epoch 41: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.3995 - loss: 1.0701 - val_accuracy: 0.3966 - val_loss: 1.0627
Epoch 42/200
357/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4007 - loss: 1.0692
Epoch 42: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.4008 - loss: 1.0692 - val_accuracy: 0.4023 - val_loss: 1.0676
Epoch 43/200
350/357 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4074 - loss: 1.0690
Epoch 43: val_loss did not improve from 1.05914
357/357 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4073 - loss: 1.0690 - val_accuracy: 0.3976 - val_loss: 1.0666
Epoch 44/200
356/357 ━━━━━━━━━━━━━━━━━━